In [1]:
# %matplotlib inline
%matplotlib ipympl

from IPython.display import clear_output

from datetime import datetime
import os
from os.path import dirname, abspath, join
from os import getcwd
import sys

import random
import string
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D


import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
#Class definitions for NN model and learning algorithm
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(N_STATES, HIDDEN_LAYER)
        nn.init.kaiming_uniform_(self.fc1.weight)
        
#         self.fc2 = nn.Linear(HIDDEN_LAYER, HIDDEN_LAYER)
#         nn.init.kaiming_uniform_(self.fc2.weight)
        
#         self.fc3 = nn.Linear(HIDDEN_LAYER, HIDDEN_LAYER)
#         nn.init.kaiming_uniform_(self.fc3.weight)

        self.fc_out = nn.Linear(HIDDEN_LAYER, N_ACTIONS)
        nn.init.xavier_uniform_(self.fc_out.weight) 
    
    def forward(self, x):
        x = self.fc1(x)
        if   (ACTIVATION==0): x = F.relu(x)
        elif (ACTIVATION==1): x = F.leaky_relu(x)
        else                : print("NN ACTIVATION: Error")
#         x = F.leaky_relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.fc3(x)
#         x = F.relu(x)
        x = self.fc_out(x)
        return x
    
class DQN(object):
    def __init__(self):
        if(GPU): 
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        else:
            device = torch.device('cpu')
        self.eval_net = Net()
        self.eval_net.to(device)
        self.device = device
        print("Neural net:")
        print(self.eval_net)

  
    def get_qvals(self,x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        x = x.to(self.device)
        actions_value = self.eval_net.forward(x)
        actions_value = actions_value.to(torch.device("cpu"))
        actions_value = actions_value.data.numpy()
        return actions_value
    
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        x = x.to(self.device)

        # input only one sample
        if True:   # greedy
            actions_value = self.eval_net.forward(x)
            actions_value = actions_value.to(torch.device("cpu"))
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] # return the argmax index
        return action

In [4]:
def stdize(s):
    MU_BATT = 0.5
    SD_BATT = 0.15
    
    MU_ENP = 0
    SD_ENP = 0.15
    
    MU_HENERGY = 0.35
    SD_HENERGY = 0.25
    
    MU_FCAST = 0.42
    SD_FCAST = 0.27
    
    norm_batt, norm_enp, norm_henergy, norm_fcast = s
    
    std_batt    = (norm_batt    - MU_BATT    )/SD_BATT
    std_enp     = (norm_enp     - MU_ENP     )/SD_ENP
    std_henergy = (norm_henergy - MU_HENERGY )/SD_HENERGY
    std_fcast   = (norm_fcast   - MU_FCAST   )/SD_FCAST

    return [std_batt, std_enp, std_henergy, std_fcast]

In [5]:
BMAX          = 10000
HMAX          = 1000
NO_OF_DAYTYPE = 10

In [6]:
# Hyper Parameters
N_ACTIONS           = 10           #no. of duty cycles (0 to 9)
N_STATES            = 4            #number of state space parameter [batt, enp, henergy, fcast]
GPU                 = False
ACTIVATION          = 0            # 0 = RELU, 1 = Leaky RELU
HIDDEN_LAYER        = 50

In [7]:
NAME = "I1"
seed_arg = 4

In [8]:
seedlist = np.array([161, 314, 228, 271828, 230, 4271031, 5526538, 6610165, 9849252, 34534, 73422, 8765])
seed = seedlist[seed_arg]
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [9]:
MODELNAME  = NAME + '_' + str(seed) + '.pt'
print("MODEL NAME: ", MODELNAME)
S_FILENAME = './models/'+ MODELNAME

MODEL NAME:  I1_230.pt


In [10]:
dqn = DQN()
dqn.eval_net.load_state_dict(torch.load(S_FILENAME))
dqn.eval_net.eval()

Neural net:
Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc_out): Linear(in_features=50, out_features=10, bias=True)
)


Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc_out): Linear(in_features=50, out_features=10, bias=True)
)

In [11]:
def get_max_qval(batt, enp, henergy, fcast):
    norm_batt    = batt    /BMAX
    norm_enp     = enp     /BMAX
    norm_henergy = henergy /HMAX
    norm_fcast   = fcast   /(NO_OF_DAYTYPE-1)
    
    state = [norm_batt, norm_enp, norm_henergy, norm_fcast]
    s = stdize(state)
    qval = dqn.get_qvals(s).flatten()
    max_qval = np.max(qval)
       
    return max_qval
    

In [12]:
def get_best_action(batt, enp, henergy, fcast):
    
    norm_batt    = batt    /BMAX
    norm_enp     = enp     /BMAX
    norm_henergy = henergy /HMAX
    norm_fcast   = fcast   /(NO_OF_DAYTYPE-1)
    
    state = [norm_batt, norm_enp, norm_henergy, norm_fcast]
    s = stdize(state)
    greedy_action = dqn.choose_greedy_action(s)
    
    return greedy_action

In [15]:
def actionmap(batt, fcast):
    plt.clf()
    print("BATT   :", batt,    end='\t\t\t\t')
    print("FCAST  :", fcast)
    
    BATT = batt
    FCAST = fcast
        
    enp     = np.linspace(-BMAX/2, BMAX/2, 100)
    henergy = np.linspace(0,       HMAX,   100)


    X, Y = np.meshgrid(enp, henergy)
    Z = vec_best_action(BATT, X, Y, FCAST)
    
    my_cmap = plt.cm.get_cmap('jet', 11)
    
    im = plt.imshow(Z, cmap=my_cmap, vmax = 9, vmin=0)
    plt.xlabel("ENP")
    plt.ylabel("Harvested Energy")

#     ax.set_title('Best Actions');
#     ax.set_xlabel("ENP");
#     im.set_ylabel("Harvested Energy");
    fig.colorbar(im)
    plt.show()
    return 0

In [16]:
batt    = widgets.IntSlider(min=0,    max=BMAX,  step=50, value=BMAX/2)
fcast   = widgets.IntSlider(min=0,max=9,step=1,value=3)
vec_best_action = np.vectorize(get_best_action)

ui = widgets.HBox([batt,fcast])
fig = plt.figure(figsize=(10,7))
out = widgets.interactive_output(actionmap, {'batt':batt, 'fcast': fcast})
display(ui, out)

FigureCanvasNbAgg()

Output()